In [5]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from netCDF4 import Dataset
from netCDF4 import num2date
from datetime import datetime, timedelta
from scipy.integrate import odeint
import os
from tqdm import tqdm

## Download files

In [6]:
import requests
from bs4 import BeautifulSoup
import os

def download_nc_files(url, download_folder):
    # Ensure download folder exists
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    # Fetch the content of the page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags, which contain the links
    links = soup.find_all('a')

    for link in links:
        href = link.get('href')
        # Check if the link ends with .nc
        if href and href.endswith('.nc'):
            # Construct the full URL for the file
            file_url = url + href if not href.startswith('http') else href
            # Download the file
            print(f"Downloading {file_url}")
            file_response = requests.get(file_url)
            # Save the file
            file_path = os.path.join(download_folder, href.split('/')[-1])
            with open(file_path, 'wb') as file:
                file.write(file_response.content)


In [7]:
# for i in range(2002, 2024):
#     url = f'https://www.ncei.noaa.gov/data/global-precipitation-climatology-project-gpcp-monthly/access/{i}/'
#     download_folder = './prec_data/'
#     download_nc_files(url, download_folder)

## Process Data

In [8]:
def is_leap_year(year):
    """Check if a year is a leap year."""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def convert_to_date(year_fraction):
    year_str, fraction_str = year_fraction.split('.')
    year = int(year_str)
    fraction = float("0." + fraction_str)
    days_in_year = 366 if is_leap_year(year) else 365
    day_of_year = int(round(fraction * days_in_year))
    date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)  # Subtract 1 because January 1 is day 1
    return date.strftime('%Y%m')

In [9]:
temp_data = pd.read_csv('./data/temp_data_2002.csv')
# ice_data = pd.read_csv('./glacier_data/data/mass_balance.csv')
with open('./data/greenland_mass_200204_202311.txt', 'r') as file:
    gl_ice = file.readlines()
with open('./data/antarctica_mass_200204_202311.txt', 'r') as file:
    antar_ice = file.readlines()

In [10]:
gl_start_index = next(i for i, line in enumerate(gl_ice) if 'HDR' not in line)
antar_start_index = next(i for i, line in enumerate(antar_ice) if 'HDR' not in line)
gl_ice_data = [[each for each in line.split(" ") if len(each) != 0] for line in gl_ice[gl_start_index:]]
antar_ice_data = [[each for each in line.split(" ") if len(each) != 0] for line in antar_ice[antar_start_index:]]
len(gl_ice_data), len(antar_ice_data)

(227, 227)

In [11]:
gl_ice_df = pd.DataFrame(gl_ice_data)
gl_ice_df.columns = ["date", "mass", "error"]
antar_ice_df = pd.DataFrame(antar_ice_data)
antar_ice_df.columns = ["date", "mass", "error"]
total_ice_df = pd.merge(gl_ice_df, antar_ice_df, how="inner", on="date")
total_ice_df = pd.DataFrame({"date": total_ice_df["date"], "mass": total_ice_df["mass_x"].astype(float) + total_ice_df['mass_y'].astype(float)})
total_ice_df['date'] = total_ice_df['date'].apply(convert_to_date)
total_ice_df

,date,mass
0,200204,0.00
1,200205,65.03
2,200208,-271.13
3,200209,-165.07
4,200210,-119.24
...,...,...
222,202307,-7700.08
223,202308,-7768.79
224,202309,-7857.56
225,202310,-7982.47


In [12]:
temp_df = temp_data[4:].rename(columns={"Global Ocean January - December Temperature Anomalies":"date"}).reset_index().drop(columns="index")
temp_df['temp'] = temp_df['temp'].astype(float)
temp_df

,date,temp
0,200201,0.46
1,200202,0.47
2,200203,0.47
3,200204,0.48
4,200205,0.48
...,...,...
259,202308,0.79
260,202309,0.82
261,202310,0.85
262,202311,0.88


In [13]:
folder_path = './data/prec_data/'
nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
all_date = []
all_prec = []
for file in nc_files:
    nc_data = Dataset(f"{folder_path}/{file}", mode='r')
    precipitation = nc_data.variables['precip'][:]
    time = nc_data.variables['time']
    units = "days since 1979-01-01"
    calendar = "standard"
    dates = num2date(time[:],time.units)[0]
    nc_data.close()
    prec = np.mean(precipitation)
    time = str(dates.year)+str(dates.month).zfill(2)
    all_date.append(time)
    all_prec.append(prec)

prec_df = pd.DataFrame({'date': all_date, 'prec': all_prec})
prec_df

,date,prec
0,201707,2.359206
1,202101,2.160409
2,200312,2.237226
3,202302,2.205895
4,202306,2.293744
...,...,...
261,200203,2.262339
262,201901,2.224894
263,201105,2.170068
264,200405,2.172355


In [14]:
complete_dates = pd.date_range(start="2002-04", end="2023-11", freq='MS')
complete_dates = [str(each).split("-")[0] + str(each).split("-")[1] for each in complete_dates]
final_df = pd.merge(temp_df, total_ice_df, how='inner', on='date')
final_df = pd.merge(final_df, prec_df, how='inner', on='date')
final_df = final_df.drop_duplicates(subset='date', keep="first")
date_df = pd.DataFrame({"date": complete_dates})
final_df = pd.merge(date_df, final_df, how='left', on='date')
final_df['date'] = final_df['date'].apply(lambda x: x[2:4] + "-" + x[4:])
final_df

,date,temp,mass,prec
0,02-04,0.48,0.00,2.126420
1,02-05,0.48,65.03,2.183341
2,02-06,NaN,NaN,NaN
3,02-07,NaN,NaN,NaN
4,02-08,0.48,-271.13,2.291364
...,...,...,...,...
255,23-07,0.77,-7700.08,2.338623
256,23-08,0.79,-7768.79,2.364702
257,23-09,0.82,-7857.56,2.383863
258,23-10,0.85,-7982.47,2.319903


In [15]:
final_df['temp'] = final_df['temp'].interpolate(method='linear')
final_df['mass'] = final_df['mass'].interpolate(method='linear')
final_df['prec'] = final_df['prec'].interpolate(method='linear')
final_df

,date,temp,mass,prec
0,02-04,0.48,0.000000,2.126420
1,02-05,0.48,65.030000,2.183341
2,02-06,0.48,-47.023333,2.219349
3,02-07,0.48,-159.076667,2.255356
4,02-08,0.48,-271.130000,2.291364
...,...,...,...,...
255,23-07,0.77,-7700.080000,2.338623
256,23-08,0.79,-7768.790000,2.364702
257,23-09,0.82,-7857.560000,2.383863
258,23-10,0.85,-7982.470000,2.319903


In [16]:
# final_df.to_csv("data/final_data.csv", index=False)